<a href="https://colab.research.google.com/github/adanog/DataScienceImagingHub/blob/main/241016_Predict_Class_using_Trained_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# About the notebook
The purpose of this Jupyter Notebook is to use a pre-trained deep learning model to generate class predictions for a given input image.

# References
[1] Hernández-Herrera et. al. 2023, https://cys.cic.ipn.mx/ojs/index.php/CyS/article/view/4772/3617

# 00 - Special Instructions for Google Colab Users

The following lines of code should be executed only when running your script on Google Colab. This is crucial to leverage the additional features provided by Colab, most notably, the availability of a free GPU.  **If, you're running the code locally, this line can be skipped (GO TO STEP 01 - Loading dependencies) as it pertains specifically to the Colab setup.**

## Give access to google drive

### If you have a shared folder just add a direct access from google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Copy code to current session

In [5]:
!git clone https://github.com/paul-hernandez-herrera/image_classification_pytorch
import os
workbookDir = "/content/image_classification_pytorch/"
os.chdir(workbookDir)

Cloning into 'image_classification_pytorch'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 202 (delta 112), reused 165 (delta 78), pack-reused 0 (from 0)
Receiving objects: 100% (202/202), 765.11 KiB | 5.67 MiB/s, done.
Resolving deltas: 100% (112/112), done.


# 01 - Loading dependencies
In this notebook, before running any code, there are several libraries and modules that need to be imported to ensure that the notebook runs smoothly. These libraries and modules contain pre-written code that performs specific tasks, such as reading and processing images, defining the UNET model, and training the model.

In [7]:
import os

# Check if the global variable 'workbookDir' is defined
if 'workbookDir' not in globals():
    print('Updating working directory')

    # Get the directory name of the current working directory
    workbookDir = os.path.dirname(os.getcwd())

    # Change the current working directory to the one above
    os.chdir(workbookDir)

# Print the current working directory
print(os.getcwd())

# Import the PyTorch library for machine learning and deep learning
import torch

# Import custom classes/functions for predictions and image display
from core_code.predict import PredictClassInteractive
from core_code.util.show_image import show_images_predicted_class_interactive

# Load the autoreload extension for Jupyter notebooks
%load_ext autoreload

# Enable autoreload for all modules before executing code
%autoreload 2


/content/image_classification_pytorch
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 01.5 - Create stacks from two channels, run only if necessary

The model require one single image from 2 channels in tif format. This cell is create single images from to brightfield and fluorescence image to a single one.

In [ ]:
pip install Pillow numpy tifffile

In [ ]:
import os
import numpy as np
import tifffile as tiff

# Paths to the directories
campo_claro_dir = '/content/drive/MyDrive/20241016_ImageClassifierFocus/Dataset/Canal campo claro/'
fluorescencia_dir = '/content/drive/MyDrive/20241016_ImageClassifierFocus/Dataset/Canal fluorescencia'

# Get all TIFF files from both directories
campo_claro_files = sorted([os.path.join(campo_claro_dir, f) for f in os.listdir(campo_claro_dir) if f.endswith('.ome.tif')])
fluorescencia_files = sorted([os.path.join(fluorescencia_dir, f) for f in os.listdir(fluorescencia_dir) if f.endswith('.ome.tif')])

# Check that we have the same number of files in both directories
if len(campo_claro_files) != len(fluorescencia_files):
    raise ValueError("Number of images in 'Canal Campo Claro' and 'Canal Fluorescencia' must match.")

# Process each pair of files
for i, (file1_path, file2_path) in enumerate(zip(campo_claro_files, fluorescencia_files)):
    # Read the images using tifffile
    image1 = tiff.imread(file1_path)
    image2 = tiff.imread(file2_path)

    # Ensure both images have the same size
    if image1.shape != image2.shape:
        raise ValueError(f"Images {file1_path} and {file2_path} must be the same size to concatenate them as channels.")

    # Convert to appropriate shape if necessary
    if image1.ndim == 3 and image1.shape[0] > 1:
        image1 = image1.transpose(1, 2, 0)  # Change from (C, H, W) to (H, W, C)
    if image2.ndim == 3 and image2.shape[0] > 1:
        image2 = image2.transpose(1, 2, 0)

    # Stack the images along a new channel dimension
    concatenated = np.stack((image1, image2), axis=-1)  # Shape will be (H, W, C, 2)

    # Convert to shape (C, H, W) for TIFF
    concatenated = np.moveaxis(concatenated, -1, 0)  # Now shape is (2, H, W)

    # Save the concatenated image
    output_path = f'/content/drive/MyDrive/20241016_ImageClassifierFocus/Dataset/mergeChannels/concatenated_image_{i+1}.tif'
    tiff.imwrite(output_path, concatenated)

    # print(f"Concatenated image saved at {output_path} with shape: {concatenated.shape}")


# 02 - Setting required parameters
In this section, users can specify the necessary parameters to predict the segmentation mask for a given input image. The following parameters are required:

**Model path**: The path to the trained model that will be used for segmentation prediction.

**Input path**: The path to the folder containing the input images, or the path to a single 'tif' image.

**Output path (Optional)**: The path where the output of the network will be saved. If you do not provide an output path, the algorithm will automatically create a folder named 'output' in the same folder as the input images, and save the predictions there.

**Device**: The device that will be used to perform the operations.

**Do not run this cell**

In [ ]:
predict_interactive = PredictClassInteractive()

Text(value='', description='Model path:', layout=Layout(flex='1 1 auto', width='auto'), placeholder='Insert pa…

Text(value='', description='Folder path:', layout=Layout(flex='1 1 auto', width='auto'), placeholder='Insert p…

Text(value='', description='Output path:', layout=Layout(flex='1 1 auto', width='auto'), placeholder='Insert p…

Dropdown(description='Device: ', options=(('CPU', 'cpu'),), style=DescriptionStyle(description_width='initial'…

# 03 - Do the prediction
This line of code allows you to predict the images using the trained deep learning model.

In [ ]:
output = predict_interactive.run()

/content/image_classification_pytorch/core_code/util/deeplearning_util.py:133: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location

/content/drive/MyDrive/20241016_ImageClassifierFocus/Models/.best_model_e14.pth


/content/image_classification_pytorch/core_code/predict.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(input_img).unsqueeze(0).to(device=device)


concatenated_image_1.tif  DONE
concatenated_image_2.tif  DONE
concatenated_image_3.tif  DONE
concatenated_image_4.tif  DONE
concatenated_image_5.tif  DONE
concatenated_image_6.tif  DONE
concatenated_image_7.tif  DONE
concatenated_image_8.tif  DONE
concatenated_image_9.tif  DONE
concatenated_image_10.tif  DONE
concatenated_image_11.tif  DONE
concatenated_image_12.tif  DONE
concatenated_image_13.tif  DONE
concatenated_image_14.tif  DONE
concatenated_image_15.tif  DONE
concatenated_image_16.tif  DONE
concatenated_image_17.tif  DONE
concatenated_image_18.tif  DONE
concatenated_image_19.tif  DONE
concatenated_image_20.tif  DONE
concatenated_image_21.tif  DONE
concatenated_image_22.tif  DONE
concatenated_image_23.tif  DONE
concatenated_image_24.tif  DONE
concatenated_image_25.tif  DONE
concatenated_image_26.tif  DONE
concatenated_image_27.tif  DONE
concatenated_image_28.tif  DONE
concatenated_image_29.tif  DONE
concatenated_image_30.tif  DONE
concatenated_image_31.tif  DONE
concatenated_imag

# 04 - Visualization
This sections provides an opportunity for the user to inspect and visually analyze the results of the segmentation prediction. This step is important to ensure that the predicted segmentations are appropriate and accurate.

In [ ]:
show_images_predicted_class_interactive(output["inputs"], output["predicted_class"])

NameError: name 'output' is not defined